So here we shall basically split the data into two parts one will be NLP part and other will be the numertic part.
I have muniplicated dataset on the basis of some assumptions that is:                                     
->id does not have any impact on the rating of the movie  
->production companies does not seem to have much impact on the rating of the movie  the logic behind it is that there are 1.5l prodcution companies and almost 7l entires. So here the categorical encoding will not be of much help. Also there are lots of null values in this columns more than 50% of the values are null.

Here I shall make 3 models one taking different columns for different models                      
-> 1st model: using the columns



## Importing libraries

In [1]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
import sys
import matplotlib.pyplot as plt
sys.path.append("../")

2022-06-30 23:21:24.663099: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
X_train = pd.read_csv('dataset/X_train.csv')
y_train = pd.read_csv('dataset/y_train.csv')

In [4]:
X_train.head()

,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_count,credits,keywords
0,Alev Alev,Drama-Thriller,tr,NaN,0.664,Erler Film,01-01-1984,0,0,118.0,Released,NaN,2,Tarık Akan-Gülşen Bubikoğlu-Cüneyt Arkın-Çiğde...,pregnancy-model-sea captain-businessman-illega...
1,Those Who Work,Drama,fr,Frank a man of action who worked his way up al...,4.174,Box Productions-Novak Prod-Office Fédéral de l...,04-10-2018,0,0,102.0,Released,NaN,67,Olivier Gourmet-Adèle Bochatay-Delphine Bibet-...,NaN
2,Driven,NaN,en,In a world of adrenaline and speed a quadriple...,0.600,NaN,13-12-2019,0,0,17.0,Released,NaN,0,NaN,NaN
3,Netherlands Documentary,NaN,en,Sex Drugs & Other Taboo Topics The World Is To...,0.600,NaN,27-02-2020,0,0,60.0,Released,NaN,0,NaN,NaN
4,Utta Danella - Der Verlobte meiner besten Freu...,Drama,de,Katharina and Elena are best friends. After a ...,1.152,NaN,03-04-2009,0,0,88.0,Released,NaN,2,Henriette Richter-Röhl-Ina Paule Klink-Robert ...,NaN


In [5]:
X_train.isnull().sum()

title                        4
genres                  223398
original_language            0
overview                122950
popularity                   0
production_companies    402472
release_date             60287
budget                       0
revenue                      0
runtime                  40194
status                       0
tagline                 641108
vote_count                   0
credits                 231217
keywords                534209
dtype: int64

## First lets work on the Numeric part

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752678 entries, 0 to 752677
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   title                 752674 non-null  object 
 1   genres                529280 non-null  object 
 2   original_language     752678 non-null  object 
 3   overview              629728 non-null  object 
 4   popularity            752678 non-null  float64
 5   production_companies  350206 non-null  object 
 6   release_date          692391 non-null  object 
 7   budget                752678 non-null  int64  
 8   revenue               752678 non-null  int64  
 9   runtime               712484 non-null  float64
 10  status                752678 non-null  object 
 11  tagline               111570 non-null  object 
 12  vote_count            752678 non-null  int64  
 13  credits               521461 non-null  object 
 14  keywords              218469 non-null  object 
dtype

In [7]:
le = preprocessing.LabelEncoder()
le.fit(list(X_train['original_language']))
org_lang = le.transform(list(X_train['original_language']))
le.fit(X_train['status'])
status = le.transform(list(X_train['status']))
X_train.drop(columns=['original_language','status'] , inplace= True)

In [8]:
X_train.insert(1, 'status', status)
X_train.insert(0, 'original_language', org_lang)
X_train.head()

,original_language,title,status,genres,overview,popularity,production_companies,release_date,budget,revenue,runtime,tagline,vote_count,credits,keywords
0,147,Alev Alev,4,Drama-Thriller,NaN,0.664,Erler Film,01-01-1984,0,0,118.0,NaN,2,Tarık Akan-Gülşen Bubikoğlu-Cüneyt Arkın-Çiğde...,pregnancy-model-sea captain-businessman-illega...
1,41,Those Who Work,4,Drama,Frank a man of action who worked his way up al...,4.174,Box Productions-Novak Prod-Office Fédéral de l...,04-10-2018,0,0,102.0,NaN,67,Olivier Gourmet-Adèle Bochatay-Delphine Bibet-...,NaN
2,31,Driven,4,NaN,In a world of adrenaline and speed a quadriple...,0.600,NaN,13-12-2019,0,0,17.0,NaN,0,NaN,NaN
3,31,Netherlands Documentary,4,NaN,Sex Drugs & Other Taboo Topics The World Is To...,0.600,NaN,27-02-2020,0,0,60.0,NaN,0,NaN,NaN
4,27,Utta Danella - Der Verlobte meiner besten Freu...,4,Drama,Katharina and Elena are best friends. After a ...,1.152,NaN,03-04-2009,0,0,88.0,NaN,2,Henriette Richter-Röhl-Ina Paule Klink-Robert ...,NaN


### Building the neural network for numeric part of the code

In [9]:
number_columns = ['vote_average', 'vote_count', 'runtime', 'revenue', 'budget', 'popularity']
X_train.isnull().sum()

original_language            0
title                        4
status                       0
genres                  223398
overview                122950
popularity                   0
production_companies    402472
release_date             60287
budget                       0
revenue                      0
runtime                  40194
tagline                 641108
vote_count                   0
credits                 231217
keywords                534209
dtype: int64

In [10]:
X_train['runtime'] = X_train['runtime'].fillna(X_train['runtime'].mean())

In [11]:
X_train.isna().sum()

original_language            0
title                        4
status                       0
genres                  223398
overview                122950
popularity                   0
production_companies    402472
release_date             60287
budget                       0
revenue                      0
runtime                      0
tagline                 641108
vote_count                   0
credits                 231217
keywords                534209
dtype: int64

In [12]:
X_train = X_train[['vote_count', 'runtime', 'revenue', 'budget', 'popularity', 'original_language', 'status']]

In [13]:
X_train.head()

,vote_count,runtime,revenue,budget,popularity,original_language,status
0,2,118.0,0,0,0.664,147,4
1,67,102.0,0,0,4.174,41,4
2,0,17.0,0,0,0.600,31,4
3,0,60.0,0,0,0.600,31,4
4,2,88.0,0,0,1.152,27,4


In [14]:
X_train.isnull().sum()

vote_count           0
runtime              0
revenue              0
budget               0
popularity           0
original_language    0
status               0
dtype: int64

In [15]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 34,177
Trainable params: 34,177
Non-trainable params: 0
_________________________________________________________________


2022-06-30 17:22:38.931433: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-30 17:22:38.933053: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-30 17:22:38.995730: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-06-30 17:22:38.996401: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:1b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-06-30 17:22:38.996955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:3d:00.0 name: Tesl

In [16]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hX_train5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [17]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [18]:
# with tf.device('/GPU:0'):
NN_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split = 0.1, callbacks=callbacks_list)

2022-06-30 17:22:44.217036: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-30 17:22:44.236101: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2900000000 Hz


Epoch 1/100


2022-06-30 17:22:44.586262: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


21170/21170 [==============================] - 136s 6ms/step - loss: 115.6467 - mean_absolute_error: 115.6467 - val_loss: 1.0921 - val_mean_absolute_error: 1.0921

Epoch 00001: val_loss improved from inf to 1.09207, saving model to Weights-001--1.09207.hX_train5


2022-06-30 17:25:00.186720: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: Weights-001--1.09207.hX_train5/assets
Epoch 2/100
21170/21170 [==============================] - 143s 7ms/step - loss: 0.6901 - mean_absolute_error: 0.6901 - val_loss: 0.8500 - val_mean_absolute_error: 0.8500

Epoch 00002: val_loss improved from 1.09207 to 0.85003, saving model to Weights-002--0.85003.hX_train5
INFO:tensorflow:Assets written to: Weights-002--0.85003.hX_train5/assets
Epoch 3/100
21170/21170 [==============================] - 130s 6ms/step - loss: 0.7004 - mean_absolute_error: 0.7004 - val_loss: 1.6740 - val_mean_absolute_error: 1.6740

Epoch 00003: val_loss did not improve from 0.85003
Epoch 4/100
21170/21170 [==============================] - 142s 7ms/step - loss: 0.7342 - mean_absolute_error: 0.7342 - val_loss: 2.3527 - val_mean_absolute_error: 2.3527

Epoch 00004: val_loss did not improve from 0.85003
Epoch 5/100
21170/21170 [==============================] - 141s 7ms/step - loss: 0.6725 - mean_absolute_error: 0.6725 - val_loss: 0.7

In [11]:
NN_model = tf.keras.models.load_model('/home/dwip.dalal/movies_rating_prediction/Weights-048--0.61986.hX_train5')

In [12]:
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 34,177
Trainable params: 34,177
Non-trainable params: 0
_________________________________________________________________
